<a href="https://colab.research.google.com/github/arturbernardo/data_stars/blob/main/Gaia2Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install pyvo
!pip install mocpy
!pip install astropy
!pip install ipyaladin
!pip install astroquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 64.2 MB/s eta 0:00:00


In [35]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt


In [39]:
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr2.gaia_source"  # Select Data Release 2
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default

In [40]:
from astroquery.gaia import Gaia
gaiadr3_table = Gaia.load_table('gaiadr3.gaia_source')
print(gaiadr3_table)


TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152


In [43]:
for column in gaiadr3_table.columns:
  print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

In [17]:
# the catalogue name in VizieR
CATALOGUE = "J/A+A/616/A10"
# CATALOGUE = "J/A+A/620/A128"


In [33]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [19]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

46 open clusters GaiaDR2 HR diagrams
Short Name: J/A+A/616/A10
IVOA Identifier: ivo://cds.vizier/j/a+a/616/a10
Access modes: conesearch, tap#aux, web
- tap#aux: http://tapvizier.cds.unistra.fr/TAPVizieR/tap
- webpage: http://vizier.cds.unistra.fr/viz-bin/VizieR-2?-source=J/A+A/616/A10
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/616/A10/tablea1a?, description: Cone search capability
 for table J/A+A/616/A10/tablea1a (Stars in nine open clusters within 250pc)
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/616/A10/tablea1b?, description: Cone search capability
 for table J/A+A/616/A10/tablea1b (Stars in 37 open clusters beyond 250pc)
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/616/A10/tablea3?, description: Cone search capability
 for table J/A+A/616/A10/tablea3 (Mean parameters for clusters within 250pc)
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/616/A10/tablea4?, description: Cone s

In [6]:
voresource.get_tables()

{'J/A+A/620/A128/hqsample': <VODataServiceTable name="J/A+A/620/A128/hqsample">... 51 columns ...</VODataServiceTable>,
 'J/A+A/620/A128/lqsample': <VODataServiceTable name="J/A+A/620/A128/lqsample">... 51 columns ...</VODataServiceTable>,
 'J/A+A/620/A128/refs': <VODataServiceTable name="J/A+A/620/A128/refs">... 5 columns ...</VODataServiceTable>}

In [20]:
tables = voresource.get_tables()
tables_names = list(tables.keys())
tables_names

['J/A+A/616/A10/tablea1a',
 'J/A+A/616/A10/tablea1b',
 'J/A+A/616/A10/tablea3',
 'J/A+A/616/A10/tablea4']

In [25]:
# get the first table of the catalogue
first_table_name = tables_names[3]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records.fielddescs

[<FIELD ID="recno" datatype="int" name="recno" ucd="meta.record"/>,
 <FIELD ID="NMemb" datatype="short" name="NMemb" ucd="meta.number"/>,
 <FIELD ID="Cluster" arraysize="*" datatype="char" name="Cluster" ucd="meta.id.parent"/>,
 <FIELD ID="RAJ2000" datatype="double" name="RAJ2000" ref="coosys_c36160010ta4" ucd="pos.eq.ra;meta.main" unit="deg"/>,
 <FIELD ID="DEJ2000" datatype="double" name="DEJ2000" ref="coosys_c36160010ta4" ucd="pos.eq.dec;meta.main" unit="deg"/>,
 <FIELD ID="plx" datatype="double" name="plx" ucd="pos.parallax.trig" unit="mas"/>,
 <FIELD ID="e_plx" datatype="double" name="e_plx" ucd="stat.error;pos.parallax" unit="mas"/>,
 <FIELD ID="pmRA" datatype="double" name="pmRA" ref="coosys_c36160010ta4" ucd="pos.pm;pos.eq.ra" unit="mas / yr"/>,
 <FIELD ID="e_pmRA" datatype="double" name="e_pmRA" ucd="stat.error;pos.pm;pos.eq.ra" unit="mas / yr"/>,
 <FIELD ID="pmDE" datatype="double" name="pmDE" ref="coosys_c36160010ta4" ucd="pos.pm;pos.eq.dec" unit="mas / yr"/>,
 <FIELD ID="e_p

In [9]:
tap_records.infos

{'QUERY_STATUS': 'OK',
 'PROVIDER': 'CDS',
 'QUERY': 'select TOP 10 * from "J/A+A/620/A128/hqsample"',
 'ivoid_service': 'ivo://CDS.VizieR/TAP',
 'publisher': 'ivo://CDS',
 'server_software': 'TAPVizieR-Vollt/1.1.3',
 'service_protocol': 'ivo://ivoa.net/std/TAP/v1.1',
 'request_date': '2025-04-06 18:55:10',
 'contact': 'cds-question@unistra.fr',
 'ivoid': 'ivo://CDS.VizieR/J/A+A/620/A128',
 'reference_url': 'https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/620/A128',
 'rights_uri': 'https://cds.unistra.fr/vizier-org/licences_vizier.html',
 'creator': 'Vioque M.',
 'article': '2018A&A...620A.128V',
 'publication_date': '2018-12-07 10:26:58+01',
 'product_type': 'Critical Comp.'}

In [22]:
print(tap_records)

<DALResultsTable length=10>
recno       Source      Cluster  RAdeg    DEdeg     Gmag    plx    e_plx    _RA      _DE   
                                  deg      deg      mag     mas     mas     deg      deg   
int32       int64        object float64  float64  float64 float64 float64 float64  float64 
----- ----------------- ------- -------- -------- ------- ------- ------- -------- --------
  254 36629890579758592  Hyades 58.14384 11.26075  12.626  28.593   0.012 58.14306 11.26079
  494 38329666836450304  Hyades 59.56042 12.62797  15.111  21.914   0.046 59.55985 12.62802
  493 38354680725946240  Hyades 58.75658 12.48554   9.766  21.715   0.015 58.75601 12.48559
  503 38670343642226816  Hyades 58.44438 13.39193  14.832  24.163   0.025 58.44372 13.39199
  504 38929141191925760  Hyades 60.18786 13.90626   13.33  21.594   0.065  60.1873 13.90633
  394 39305036729495936  Hyades   60.926 14.99122  15.039    24.1   0.041 60.92537 14.99133
  495 40470720915966208  Hyades 58.72235 16.31552  1

NameError: name 'pd' is not defined

ˊ
 **ndice** de cor=BP−RP
Este índice de cor, que pode ser positivo ou negativo, dá uma indicação do tipo de cor da estrela. Estrelas mais azuis têm um índice de cor negativo (porque o BP será menor do que o RP), enquanto estrelas mais vermelhas terão um índice de cor positivo (porque o RP será maior do que o BP).

Índice de cor negativo → Estrela azul.

Índice de cor positivo → Estrela vermelha.

Por exemplo:

Estrelas azuis: Têm um índice de cor BP - RP negativo, o que significa que elas têm uma magnitude maior na banda azul (BP) do que na banda vermelha (RP).

Estrelas vermelhas: Têm um índice de cor BP - RP positivo, o que indica que elas são mais brilhantes na banda vermelha.

phot_proc_mode : Photometry processing mode (byte)

This flag indicates the photometric calibration process used for the source. The process is determined by the availability of colour information derived from the internally calibrated mean BP and RP source spectra. The following values are defined for Gaia DR3:

• 0: this corresponds to the ‘gold’ photometric dataset. Sources in this dataset have complete colour information.
• 1: this corresponds to the ‘silver’ photometric dataset. Sources in this dataset have incomplete colour information and therefore were calibrated using an iterative process that estimated the missing colour information from the source mean G and either BP or RP photometry (depending on which band had full colour information available) using empirical relationships derived from the gold dataset.
• 2: this corresponds to the ‘bronze’ photometric dataset. Sources in this dataset had insufficient colour information and therefore were calibrated using default colour information derived from the gold dataset.

bp_rp : BP - RP colour (float, Magnitude[mag])



phot_g_mean_mag : G-band mean magnitude (float, Magnitude[mag])

Mean magnitude in the G band. This is computed from the G-band mean flux (phot_g_mean_flux ) applying the magnitude zero-point in the Vega scale (see Section 5.4.1).

No error is provided for this quantity as the error distribution is only symmetric in flux space. This converts to an asymmetric error distribution in magnitude space which cannot be represented by a single error value.

BY CHATGPT:

3. Exemplo de campos relacionados à cor no Gaia DR2:
Aqui estão alguns campos do Gaia DR2 que podem ser relevantes:

BP_mag: A magnitude da estrela na banda azul (BP).

RP_mag: A magnitude da estrela na banda vermelha (RP).

G_mag: A magnitude na banda G (visível).

BP-RP: O índice de cor que calcula a diferença entre as magnitudes BP e RP.